In [1]:
%matplotlib inline

import collections
import os

import nengo
import nengo_dl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import uproot

try:
    import requests

    has_requests = True
except ImportError:
    has_requests = False

import nengo_loihi

scale = 10

train_outdir = "./train_examples/"
os.makedirs(train_outdir, exist_ok=True)

2024-04-24 23:30:45.139322: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-24 23:30:48.116081: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
from sklearn.metrics import confusion_matrix

# This function prints and plots the confusion matrix. Normalization can be applied by setting `normalize=True`.
def plot_confusion_matrix(y_true, y_pred, classes, normalize=False,  title=None, cmap=plt.cm.Blues, savename="./cm.pdf"):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
        if not title: title = 'Normalized confusion matrix'
    else:
        print('Confusion matrix, without normalization')
        if not title: title = 'Confusion matrix, without normalization'

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    ax.set(xticks=np.arange(cm.shape[1]), yticks=np.arange(cm.shape[0]), xticklabels=classes, yticklabels=classes,
           title=title, ylabel='True label', xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt), ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()

    plt.savefig(savename)
    plt.gcf().clear()
    return ax

In [3]:
# set up training data
variables = [
    'njets', 'nbjets', 'ncjets', 'nElectron', 'nMuon', 'MET_met', 'HT', # 'nLepton', 'MET_px', 'MET_py',
    'Lepton_pt', 'Lepton_eta', 'Lepton_e',# 'Lepton_phi',
    'Jet1_pt', 'Jet1_eta', 'Jet1_e', 'Jet1_btag', 'Jet2_pt', 'Jet2_eta', 'Jet2_e', 'Jet2_btag',# 'Jet_phi1', 'Jet_phi2',
    'Jet3_pt', 'Jet3_eta', 'Jet3_e', 'Jet3_btag', 'Jet4_pt', 'Jet4_eta', 'Jet4_e', 'Jet4_btag',# 'Jet_phi3', 'Jet_phi4',
    #'bjet1_pt', 'bjet1_eta', 'bjet1_e', 'bjet2_pt', 'bjet2_eta', 'bjet2_e',# 'bjet1_phi', 'bjet2_phi',
    'selbjet1_pt', 'selbjet1_eta', 'selbjet1_e', 'selbjet2_pt', 'selbjet2_eta', 'selbjet2_e',# 'selbjet1_phi', 'selbjet2_phi',

    'bbdR',   'bbdEta',   'bbdPhi',   'bbPt',   'bbEta',   'bbMass',   'bbHt',   'bbMt',  # 'bbPhi',
    'nub1dR', 'nub1dEta', 'nub1dPhi', 'nub1Pt', 'nub1Eta', 'nub1Mass', 'nub1Ht', 'nub1Mt',# 'nub1Phi',
    'nub2dR', 'nub2dEta', 'nub2dPhi', 'nub2Pt', 'nub2Eta', 'nub2Mass', 'nub2Ht', 'nub2Mt',# 'nub2Phi',
    'nubbdR', 'nubbdEta', 'nubbdPhi', 'nubbPt', 'nubbEta', 'nubbMass', 'nubbHt', 'nubbMt',# 'nubbPhi',
    'lb1dR',  'lb1dEta',  'lb1dPhi',  'lb1Pt',  'lb1Eta',  'lb1Mass',  'lb1Ht',  'lb1Mt', # 'lb1Phi',
    'lb2dR',  'lb2dEta',  'lb2dPhi',  'lb2Pt',  'lb2Eta',  'lb2Mass',  'lb2Ht',  'lb2Mt', # 'lb2Phi',
    'lbbdR',  'lbbdEta',  'lbbdPhi',  'lbbPt',  'lbbEta',  'lbbMass',  'lbbHt',  'lbbMt', # 'lbbPhi',
    'Wjb1dR', 'Wjb1dEta', 'Wjb1dPhi', 'Wjb1Pt', 'Wjb1Eta', 'Wjb1Mass', 'Wjb1Ht', 'Wjb1Mt',# 'Wjb1Phi',
    'Wjb2dR', 'Wjb2dEta', 'Wjb2dPhi', 'Wjb2Pt', 'Wjb2Eta', 'Wjb2Mass', 'Wjb2Ht', 'Wjb2Mt',# 'Wjb2Phi',
    'Wlb1dR', 'Wlb1dEta', 'Wlb1dPhi', 'Wlb1Pt', 'Wlb1Eta', 'Wlb1Mass', 'Wlb1Ht', 'Wlb1Mt',# 'Wlb1Phi',
    'Wlb2dR', 'Wlb2dEta', 'Wlb2dPhi', 'Wlb2Pt', 'Wlb2Eta', 'Wlb2Mass', 'Wlb2Ht', 'Wlb2Mt',# 'Wlb2Phi',
 
]
class_names = ["tthbb", "ttbb", "ttbj", "ttcc", "ttlf"]
nClass, nVariables = len(class_names), len(variables)

pd_data = pd.read_hdf('samples/data.h5', key='df', mode='r')
print (pd_data)
print (pd_data.columns)

train_data = pd_data.filter(items = variables)
train_data = (train_data - train_data.min())*scale/(train_data.max() - train_data.min())
train_data = np.array(train_data).astype(float)
train_out = np.array(pd_data.filter(items = ["category"])).reshape((len(pd_data),))

print (train_out)

trainlen = 267767
train_images = train_data[:trainlen, 0::]
train_labels = train_out[:trainlen]
test_images = train_data[trainlen:, 0::]
test_labels = train_out[trainlen:]

FileNotFoundError: File samples/data.h5 does not exist

In [ ]:
inp = tf.keras.Input(nVariables, name="input")

# transform input signal to spikes using trainable 1x1 convolutional layer
to_spikes_layer = tf.keras.layers.Dense(units=nVariables, activation=tf.nn.relu, name="to-spikes")
to_spikes = to_spikes_layer(inp)

# on-chip dense layers
dense0_layer = tf.keras.layers.Dense(units=256, activation=tf.nn.relu, name="dense0")
dense0 = dense0_layer(to_spikes)
dense1_layer = tf.keras.layers.Dense(units=256, activation=tf.nn.relu, name="dense1")
dense1 = dense1_layer(dense0)
dense2_layer = tf.keras.layers.Dense(units=256, activation=tf.nn.relu, name="dense2")
dense2 = dense2_layer(dense1)
# dense3_layer = tf.keras.layers.Dense(units=100, activation=tf.nn.relu, name="dense3")
# dense3 = dense3_layer(dense2)
# dense4_layer = tf.keras.layers.Dense(units=100, activation=tf.nn.relu, name="dense4")
# dense4 = dense4_layer(dense3)
# dense5_layer = tf.keras.layers.Dense(units=100, activation=tf.nn.relu, name="dense5")
# dense5 = dense5_layer(dense4)

# since this final output layer has no activation function,
# it will be converted to a `nengo.Node` and run off-chip
out_layer = tf.keras.layers.Dense(units=nClass, name="out")
out = out_layer(dense2)

model = tf.keras.Model(inputs=inp, outputs=out)
model.summary()

In [ ]:
epochs = 100
dt = 0.001  # simulation timestep
presentation_time = 0.1  # input presentation time
max_rate = 100  # neuron firing rates
# neuron spike amplitude (scaled so that the overall output is ~1)
amp = 1 / max_rate
activation = nengo.SpikingRectifiedLinear()
#activation = nengo_loihi.neurons.LoihiSpikingRectifiedLinear()
params_file = train_outdir+"./keras_to_loihi_dense"
converter = nengo_dl.Converter(model, swap_activations={tf.nn.relu: activation})

with nengo_dl.Simulator(converter.net, seed=0, minibatch_size=200) as sim:
    sim.compile(
        optimizer=tf.optimizers.RMSprop(0.001),
        loss={converter.outputs[out]: tf.losses.SparseCategoricalCrossentropy(from_logits=True)},
        metrics={converter.outputs[out]: tf.metrics.sparse_categorical_accuracy},
    )
    sim.fit(
        {converter.inputs[inp]: train_images},
        {converter.outputs[out]: train_labels},
        epochs=epochs,
    )

    # save the parameters to file
    sim.save_params(params_file)
    sim.freeze_params(converter.net)

In [ ]:
def is_spiking_type(neuron_type):
    return isinstance(neuron_type, (nengo.LIF, nengo.SpikingRectifiedLinear))

n_test, n_steps, n_plots = 10000, 30, 30

savename = train_outdir+"SpikingNeurons_"
# convert the keras model to a nengo network
nengo_converter = nengo_dl.Converter(
    model,
    scale_firing_rates=max_rate,
    swap_activations={tf.nn.relu: activation},
    synapse=0.005,
)

# get input/output objects
nengo_input = nengo_converter.inputs[inp]
nengo_output = nengo_converter.outputs[out]

# add probes to layers to record activity
with nengo_converter.net:
    probes = collections.OrderedDict(
        [
            [to_spikes_layer, nengo.Probe(nengo_converter.layers[to_spikes])],
            [dense0_layer, nengo.Probe(nengo_converter.layers[dense0])],
            [dense1_layer, nengo.Probe(nengo_converter.layers[dense1])],
            [dense2_layer, nengo.Probe(nengo_converter.layers[dense2])],
            # [dense3_layer, nengo.Probe(nengo_converter.layers[dense3])],
            # [dense4_layer, nengo.Probe(nengo_converter.layers[dense4])],
            # [dense5_layer, nengo.Probe(nengo_converter.layers[dense5])],
            #[out, nengo.Probe(nengo_converter.layers[out])],
        ]
    )

In [ ]:
# repeat inputs for some number of timesteps
n_test = 200
tiled_test_images = np.tile(test_images[:n_test], (1, n_steps, 1))
#tiled_test_images = np.tile(test_images[:], (1, n_steps, 1))

# # set some options to speed up simulation
# with nengo_converter.net:
#     nengo_dl.configure_settings(stateful=False)

# build network, load in trained weights, run inference on test images
with nengo_dl.Simulator(
    nengo_converter.net, minibatch_size=200, # progress_bar=False
) as sim:
    data = sim.predict({nengo_input: tiled_test_images})
    sim.compile(loss={nengo_converter.outputs[out]: tf.losses.SparseCategoricalCrossentropy(from_logits=True)})
    print("accuracy w/ synaps: %.2f%%"
        % sim.evaluate(test_images, {nengo_converter.outputs[out]: test_labels}, verbose=0)["loss"]
    )

# compute accuracy on test data, using output of network on
# last timestep
test_predictions = np.argmax(data[nengo_output][:, -1], axis=-1)
print("Test accuracy: %.2f%%" % (100 * np.mean(test_predictions == test_labels[:n_test, 0, 0])))
# print("Test accuracy: %.2f%%" % (100 * np.mean(test_predictions == test_labels[:, 0, 0])))

In [ ]:
# plot the results
mean_rates = []
for i in range(n_plots):
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 3, 1)
    plt.title(test_labels[i, 0])
    #plt.imshow(test_images[i, 0].reshape((7, 9)), cmap="gray")
    plt.axis("off")

    n_layers = len(probes)
    mean_rates_i = []
    for j, layer in enumerate(probes.keys()):
        #print (j, layer)
        probe = probes[layer]
        plt.subplot(n_layers, 3, (j * 3) + 2)
        plt.suptitle("Neural activities")

        outputs = data[probe][i]

        # look at only at non-zero outputs
        nonzero = (outputs > 0).any(axis=0)
        outputs = outputs[:, nonzero] if sum(nonzero) > 0 else outputs

        # undo neuron amplitude to get real firing rates
        outputs /= nengo_converter.layers[layer].ensemble.neuron_type.amplitude

        rates = outputs.mean(axis=0)
        mean_rate = rates.mean()
        mean_rates_i.append(mean_rate)
        #print('"%s" mean firing rate (example %d): %0.1f' % (layer.name, i, mean_rate))

        if is_spiking_type(activation):
            outputs *= 0.001
            plt.ylabel("# of Spikes")
        else:
            plt.ylabel("Firing rates (Hz)")

        # plot outputs of first 100 neurons
        plt.plot(outputs[:, :100])

    mean_rates.append(mean_rates_i)

    plt.xlabel("Timestep")

    plt.subplot(1, 3, 3)
    plt.title("Output predictions")
    plt.plot(tf.nn.softmax(data[nengo_output][i]))
    plt.legend(class_names, loc="upper left")
    plt.xlabel("Timestep")
    plt.ylabel("Probability")

    plt.tight_layout()
    plt.savefig(savename+str(i)+".png")

# take mean rates across all plotted examples
mean_rates = np.array(mean_rates).mean(axis=0)

In [ ]:
n_presentations = 100

# if running on Loihi, increase the max input spikes per step
hw_opts = dict(snip_max_spikes_per_step=max_rate)
with nengo_loihi.Simulator(
    nengo_converter.net,
    dt=dt,
    precompute=False,
    hardware_options=hw_opts,
) as sim:
    # run the simulation on Loihi
    sim.run(n_presentations * presentation_time)

    # check classification accuracy
    step = int(presentation_time / dt)
    output = sim.data[nengo_converter.outputs[out]][step - 1 :: step]

    acc = 100 * np.mean(
        np.argmax(output, axis=-1) == test_labels[:n_presentations, -1, 0]
    )
    print("loihi accuracy: %.2f%%" % acc)

    predicted = np.argmax(output, axis=-1)
    correct = test_labels[:n_presentations, -1, 0]

    predicted = np.array(predicted, dtype=int)
    correct = np.array(correct, dtype=int)

    print("Predicted labels:\t", predicted)
    print("Correct labels: \t", correct)
    print("loihi acc: %.2f%%" % acc)

    np.set_printoptions(precision=2)


    plot_confusion_matrix(correct, predicted, classes=class_names, 
                          savename=train_outdir+"/confusion_matrix_val.pdf")
    plot_confusion_matrix(correct, predicted, classes=class_names, normalize=True, 
                          savename=train_outdir+"/norm_confusion_matrix_val.pdf")